In [39]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

d= load_digits()

In [40]:
df= pd.DataFrame(d.data, columns= d.feature_names)
df['target']= d.target
df
Y= df.target
X= df.drop(['target'], axis= 'columns')

In [41]:
def getscore(model, xtrn,xtst, ytrn, ytst):
    model.fit(xtrn, ytrn)
    return model.score(xtst, ytst)

In [54]:
import numpy as np

score={}
models={'Random Forest':RandomForestClassifier(),'Decision Tree': DecisionTreeClassifier(),'SVM': SVC(),
        'Logistic Regression': LogisticRegression(solver='lbfgs',max_iter=2500), 'Gaussian Baive Bayes' : GaussianNB()}
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kf= KFold(n_splits=5)
for a, b in models.items():
    
    for train_index, test_index in kf.split(X):
        xtrain,xtest,ytrain,ytest = d.data[train_index], d.data[test_index], Y[train_index], Y[test_index]
        score.update( {a :getscore(models[a],xtrain, xtest, ytrain, ytest)})
      


score


C:\Python3.10\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Random Forest': 0.9220055710306406,
 'Decision Tree': 0.7910863509749304,
 'SVM': 0.9415041782729805,
 'Logistic Regression': 0.8997214484679665,
 'Gaussian Baive Bayes': 0.8133704735376045}

In [55]:
dict={}
for a, b in models.items():
    dict[a] = cross_val_score(models[a], X, Y, cv= 5)

dict

C:\Python3.10\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Random Forest': array([0.91944444, 0.91111111, 0.96100279, 0.9637883 , 0.91643454]),
 'Decision Tree': array([0.78611111, 0.70277778, 0.79108635, 0.83286908, 0.80501393]),
 'SVM': array([0.96111111, 0.94444444, 0.98328691, 0.98885794, 0.93871866]),
 'Logistic Regression': array([0.925     , 0.87777778, 0.93871866, 0.93593315, 0.89693593]),
 'Gaussian Baive Bayes': array([0.78055556, 0.78333333, 0.79387187, 0.8718663 , 0.80501393])}

In [56]:
# Now Having leanrt That that cross_val_score does exactly the same thing Now its time for Parameter Tuning

performance={}
SVM= []
Rand= []
decision=[]
c=[10, 20, 30]
k= ['linear', 'poly', 'rbf']
for a in k:
    for b in c:
        performance[str(a)+"_"+str(b) ]= np.average(cross_val_score(SVC(C=b,kernel=a), X, Y, cv= 5))
performance



{'linear_10': 0.9476973073351903,
 'linear_20': 0.9476973073351903,
 'linear_30': 0.9476973073351903,
 'poly_10': 0.96884246363355,
 'poly_20': 0.96884246363355,
 'poly_30': 0.96884246363355,
 'rbf_10': 0.9738502011761063,
 'rbf_20': 0.9738502011761063,
 'rbf_30': 0.9738502011761063}

In [57]:
#Now what if we have to chose for many folds of dataset, then for different parameters of a classifier
#and then in between different  classfiers is not it confusing?? YES, but dont worry. We have Grid Seach
# or grid search cross validation

from sklearn.model_selection import GridSearchCV
grid= GridSearchCV(SVC(), {'C': [10, 20, 30], 'kernel': ['linear', 'poly', 'rbf']}, cv=5)
                   
grid.fit(X, Y)
grid.cv_results_

{'mean_fit_time': array([0.0393271 , 0.04374347, 0.07355628, 0.04686351, 0.05086412,
        0.08181486, 0.04374347, 0.06574044, 0.10213704]),
 'std_fit_time': array([0.01030633, 0.00624959, 0.00619107, 0.00986894, 0.00729119,
        0.01452679, 0.00625032, 0.01816208, 0.02042442]),
 'mean_score_time': array([0.01558447, 0.01562777, 0.04875002, 0.01127901, 0.0156981 ,
        0.05446367, 0.01563134, 0.01196575, 0.04701605]),
 'std_score_time': array([8.65541538e-05, 1.05120696e-06, 1.63773016e-02, 6.11687331e-03,
        1.42276702e-04, 1.60345110e-02, 5.79823004e-06, 1.16064945e-02,
        1.61084446e-02]),
 'param_C': masked_array(data=[10, 10, 10, 20, 20, 20, 30, 30, 30],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'poly', 'rbf', 'linear', 'poly', 'rbf',
                    'linear', 'poly', 'rbf'],
              mask=[False,

In [58]:
df2= pd.DataFrame(grid.cv_results_)
df2[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.947697
1,10,poly,0.968842
2,10,rbf,0.973850
3,20,linear,0.947697
4,20,poly,0.968842
5,20,rbf,0.973850
6,30,linear,0.947697
7,30,poly,0.968842
8,30,rbf,0.973850


In [31]:
grid.best_score_

0.9738502011761063

In [33]:
grid.best_params_

{'C': 10, 'kernel': 'rbf'}

In [87]:
for x,y in models.items():
    print(y)

RandomForestClassifier()
DecisionTreeClassifier()
SVC()
LogisticRegression(max_iter=2500)
GaussianNB()


In [102]:
Model= { 
    'svm':{ 'model': SVC(),
            'params': {'C': [10, 20, 30], 'kernel':['poly', 'rbf'] }},
    'Random Forest': { 'model': RandomForestClassifier(),
                       'params': {'n_estimators':[20, 30, 50], 'criterion':["gini", "entropy"] }},
    'Decision Tree' : { 'model': DecisionTreeClassifier(), 
                        'params': {'criterion':["gini", "entropy"],'splitter' :["best", "random"]}},
'Logistic Regression': {'model' :LogisticRegression(max_iter=1000), 
                        'params': {'penalty' : ['l1', 'l2', 'elasticnet', 'none'], 
                                   'solver' : ['newton-cg', 'lbfgs', 'liblinear'] }}
    
}


In [104]:
Scores=[]
for key, value in Model.items():
    grid= GridSearchCV(value['model'], value['params'], cv=3, return_train_score=False)
    grid.fit(X,Y)
    Scores.append({
    'Model': key,
    'Best Scores': grid.best_score_, 
    'Best parameter': grid.best_params_
                   
    })

Scores

C:\Python3.10\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python3.10\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_re

[{'Model': 'svm',
  'Best Scores': 0.9738452977184195,
  'Best parameter': {'C': 10, 'kernel': 'rbf'}},
 {'Model': 'Random Forest',
  'Best Scores': 0.9343350027824151,
  'Best parameter': {'criterion': 'gini', 'n_estimators': 50}},
 {'Model': 'Decision Tree',
  'Best Scores': 0.8063439065108513,
  'Best parameter': {'criterion': 'entropy', 'splitter': 'random'}},
 {'Model': 'Logistic Regression',
  'Best Scores': 0.9287701725097385,
  'Best parameter': {'penalty': 'l2', 'solver': 'newton-cg'}}]

In [105]:
DF= pd.DataFrame(Scores, columns=['Model', 'Best Scores', 'Best parameter'] )
DF

,Model,Best Scores,Best parameter
0,svm,0.973845,"{'C': 10, 'kernel': 'rbf'}"
1,Random Forest,0.934335,"{'criterion': 'gini', 'n_estimators': 50}"
2,Decision Tree,0.806344,"{'criterion': 'entropy', 'splitter': 'random'}"
3,Logistic Regression,0.928770,"{'penalty': 'l2', 'solver': 'newton-cg'}"
